<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%EB%B2%88%EC%97%AD%EA%B8%B0(%EC%A0%95%EB%B0%A9%ED%96%A5%ED%95%9C%EA%B5%AD_%EC%98%81%EC%96%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torchvision

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
import torch
import torch.nn as nn
import torch.optim as optim


from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 8.2MB/s 
     |████████████████████████████████| 3.8MB 41.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6

In [4]:
from konlpy.tag import Mecab
mecab = Mecab()

mecab_kor = mecab
spacy_en = spacy.load('en')

In [5]:
mecab_kor.morphs('나는 한국인 입니다')

['나', '는', '한국인', '입니다']

In [6]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [7]:
def tokenize_en(text):#영어 단어 토큰화 후 리스트로
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_kor(text):# 한국어 단어 토큰화 후 리스트로
    return mecab_kor.morphs(text)

In [8]:
tokenize_en('I am a boy')

['I', 'am', 'a', 'boy']

In [9]:
tokenize_kor('나는 한국인 입니다')

['나', '는', '한국인', '입니다']

In [10]:
SRC = Field(tokenize = tokenize_kor, #Field: 전처리를 해주는 파이프라인 같은 개념. 앞뒤에 sos, eos 넣어주고 소문자화(한국어에선 lower필요없음)
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [11]:
import pandas as pd#데이터 불러오고, train, val, test로 나누기. tsv로 저장해야 후에 문장별로 구분이 잘 됨.
news_df = pd.read_excel('/content/drive/My Drive/B반/data/kor.xlsx', sheet_name='Sheet1') 
train_df, val_df, test_df = news_df.iloc[:50000, 1:], news_df.iloc[50000:63000, 1:], news_df.iloc[63000:, 1:]

train_df.to_csv('/content/drive/My Drive/Seq to Seq/train_df.tsv', sep='\t', index=False)
test_df.to_csv('/content/drive/My Drive/Seq to Seq/test_df.tsv', sep='\t', index=False)
val_df.to_csv('/content/drive/My Drive/Seq to Seq/val_df.tsv', sep='\t', index=False)

In [12]:
!cat '/content/drive/My Drive/Seq to Seq/train_df.tsv'

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
오늘하고 내일도 못 갈 거 같아요.	I don't think I can go there today or tomorrow.
그의 개인적인 시간을 위해 그것은 조금 곤란할 것 같아요.	I think that won't be suitable for his private time.
여기 지리적인 특징인 것 같아요.	I guess it is a geological characteristic.
아름다워서 인기가 좋을 것 같아요.	It would be popular for its beauty.
변경됀 디자인 검토안은 첨부드리는 화일과 같아요.	The reviews of the changed design are attached.
당신은 회사인데 내가 방해하는 것 같아요.	It's seems like I'm interrupting you because you're at work.
당신의 곰 인형과 내가 같이 있다면 더 좋을 것 같아요.	If your teddy bear and I are together, it will be better.
벚꽃이 더 인기 있는 것 같아요.	I think cherry blossoms are more popular.
요란 다 다이어트하면 완전 이쁠 것 같아요.	You will be gorgeous if you burn out some calories Yolanda.
머리 스타일이 나쁘지는 않은 것 같아요.	I don't think the hairstyle is bad at all.
어제 스티븐이 수업을 못 한 것 같아요.	I think Steven could not do lecture yesterday.
네, 가급적이면 평일에 가시는 게 좋을 것 같아요.	Yes, it will be better to visit on the weekdays.
영국 남자들이 참 묵둑둑하면서 자상한거 같아요.	English men are brusque but also attentive.
눈에 안 보이는 데

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
from torchtext.data import TabularDataset
#파이토치에서 사용하는 형태의 데이터셋으로 형변환 시켜주기. 전처리까지 완료 후 각각 할당해 줌.
train_data, valid_data, test_data = TabularDataset.splits(path='.',
					train='/content/drive/My Drive/Seq to Seq/train_df.tsv',
					validation='/content/drive/My Drive/Seq to Seq/val_df.tsv',
					test='/content/drive/My Drive/Seq to Seq/test_df.tsv',
					format='tsv', 
					fields=[('src', SRC), ('trg', TRG)], skip_header=True)

In [14]:
valid_data #데이터셋으로 형변환 된것을 알 수 있다.

In [15]:
print(f"Number of training examples: {len(train_data.examples)}")#갯수확인
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 50000
Number of validation examples: 13000
Number of testing examples: 12000


In [16]:
print(vars(train_data.examples[0]))#한국어 정방향으로, 영어도 정방향으로 잘 넣어줬는지 확인.
print(vars(valid_data.examples[0]))
print(vars(test_data.examples[0]))

{'src': ['나', '는', '매일', '저녁', '배트', '를', '만나', '러', '다락방', '으로', '가요', '.'], 'trg': ['i', 'go', 'to', 'the', 'attic', 'every', 'evening', 'to', 'meet', 'bat', '.']}
{'src': ['내', '가', '이번', '여름', '에', '원하', '는', '꿈', '의', '계획', '은', '세계', '여행', '이', '에요', '.'], 'trg': ['for', 'this', 'summer', ',', 'i', 'have', 'a', 'dream', 'to', 'travel', 'abroad', '.']}
{'src': ['저', '는', '오늘', '그', '이유', '에', '대해', '말', '하', '고자', '해요', '.'], 'trg': ['and', 'i', "'m", 'here', 'to', 'explain', 'the', 'reason', 'for', 'this', '.']}


In [17]:
SRC.build_vocab(train_data, min_freq = 2)#사전을 만드는데 min_freq를 통해 1번만 등장한 단어는 버린다.
TRG.build_vocab(train_data, min_freq = 2)

In [18]:
print(f"Unique tokens in source (kr) vocabulary: {len(SRC.vocab)}")#사전 내 단어의 갯수 확인. 
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (kr) vocabulary: 10064
Unique tokens in target (en) vocabulary: 8461


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #GPU 쓰라고 하는 명령

In [20]:
BATCH_SIZE = 128

In [21]:
train_iterator  = BucketIterator(#비슷한 길이의 문장끼리 하나의 배치로 묶어주라는 함수
    train_data, 
    batch_size = BATCH_SIZE, 
    device = device)


In [22]:
valid_iterator  = BucketIterator(
    valid_data, 
    batch_size = BATCH_SIZE, 
    device = device)


In [23]:
test_iterator  = BucketIterator(
    test_data, 
    batch_size = BATCH_SIZE, 
    device = device)


In [24]:
class Encoder(nn.Module):#임베딩 후 드랍아웃을 하고, RNN을 적용해주는 인코딩 클래스
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)#히든, 셀을 받아줌
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell#히든셀이 출력됨

In [25]:
class Decoder(nn.Module):#배치사이즈, 히든, 셀을 받아 디코딩 하는 클래스
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)#0번째 자리에 새로운 차원 삽입
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))#예측한 문장 출력
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [26]:
class Seq2Seq(nn.Module):#인코더와 디코더, gpu활용
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        #인코더, 디코더의 차원과 레이어 같게 해야한다는 조건
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):#teacher forcing이란, training에서 실제 정답을 보여주며 decoder를 학습시키는 것
        '''
        중요한 점은 훈련 시에는 디코더의 입력으로 
        이전 time-step의 디코더 출력값이 아닌, 정답 $Y$ 가 들어간다는 것입니다. 
        하지만 추론할 때는 정답 $Y$ 를 모르기 때문에, 
        이전 time-step에서 계산되어 나온 $\hat{y}_{t-1}$ 를 디코더의 입력으로 사용합니다. 
        이렇게 입력을 넣어주는 훈련 방법을 teacher forcing이라고 합니다.
        '''
            
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) #가장 높은 확률 예측단어(문장) 뽑아내기
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [27]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [28]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        #논문처럼 파라미터를 -0.08과 0.08사이로 맞춰주기
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10064, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(8461, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=8461, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [29]:
def count_parameters(model):#파라미터 숫자 세주기
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,439,309 trainable parameters


In [30]:
optimizer = optim.Adam(model.parameters())#아담으로 하지말고 SGD로 해보자

In [31]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]#타겟(영어) 패딩해주고

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
#기준으로 삼아서 트레인 채점할 때 사용

In [32]:
def train(model, iterator, optimizer, criterion, clip):# 테스트 후 loss를 리턴
    
    model.train() 
    
    epoch_loss = 0 
    
    for i, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()#gradient 초기화
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)#타겟과아웃풋 비교후 로스로 입력
        
        loss.backward()#역전파 계산
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #gradient clipping->norm을 수정해가면서 발산 막아주기->논문방식 적용가능!
        #아래 링크 사진확인
        #https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-6/05-gradient-clipping
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [33]:
def evaluate(model, iterator, criterion):#평가 함수, 여기선 teacher forcing안쓰고 바로 맞추게 비교하면서 한다.
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [34]:
def epoch_time(start_time, end_time):#한 에폭에 얼마나 시간이 걸렸는지 보여줌 케라스verbose역할로 쓰는 함수.
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [35]:
N_EPOCHS = 20
CLIP = 5#논문처럼 5로 맞춰줌. SGD가 적용되지 않아 큰 영향을 주지는 못할 듯.

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 37s
	Train Loss: 5.357 | Train PPL: 212.029
	 Val. Loss: 5.567 |  Val. PPL: 261.539
Epoch: 02 | Time: 1m 36s
	Train Loss: 4.963 | Train PPL: 142.976
	 Val. Loss: 5.394 |  Val. PPL: 219.997
Epoch: 03 | Time: 1m 36s
	Train Loss: 4.686 | Train PPL: 108.375
	 Val. Loss: 5.183 |  Val. PPL: 178.228
Epoch: 04 | Time: 1m 36s
	Train Loss: 4.457 | Train PPL:  86.241
	 Val. Loss: 5.109 |  Val. PPL: 165.503
Epoch: 05 | Time: 1m 36s
	Train Loss: 4.270 | Train PPL:  71.523
	 Val. Loss: 4.988 |  Val. PPL: 146.592
Epoch: 06 | Time: 1m 36s
	Train Loss: 4.086 | Train PPL:  59.475
	 Val. Loss: 4.888 |  Val. PPL: 132.678
Epoch: 07 | Time: 1m 36s
	Train Loss: 3.905 | Train PPL:  49.668
	 Val. Loss: 4.776 |  Val. PPL: 118.649
Epoch: 08 | Time: 1m 36s
	Train Loss: 3.739 | Train PPL:  42.035
	 Val. Loss: 4.747 |  Val. PPL: 115.208
Epoch: 09 | Time: 1m 36s
	Train Loss: 3.592 | Train PPL:  36.291
	 Val. Loss: 4.628 |  Val. PPL: 102.294
Epoch: 10 | Time: 1m 36s
	Train Loss: 3.458 | Train PPL

In [36]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.561 | Test PPL:  95.659 |


In [37]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.558 | Test PPL:  95.394 |


#번역


In [39]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

In [40]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                       rotation=45)
    ax.set_yticklabels(['']+translation)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

In [41]:
example_idx = 12

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['동물', '병원', '도', '있', '기', '때문', '에', '강아지', '들', '을', '데리', '고', '도', '자주', '가요', '.']
trg = ['it', 'has', 'an', 'animal', 'hospital', ',', 'so', 'i', 'took', 'puppies', 'there', '.']


In [42]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

TypeError: ignored

In [ ]:
display_attention(src, translation, attention)

In [ ]:
example_idx = 14

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

display_attention(src, translation, attention)

In [ ]:
example_idx = 18

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

display_attention(src, translation, attention)

#블루스코어


In [49]:
!pip install torchtext==0.5.0


     |████████████████████████████████| 81kB 4.2MB/s 
  Found existing installation: torchtext 0.6.0
    Uninstalling torchtext-0.6.0:
      Successfully uninstalled torchtext-0.6.0


In [55]:
from torchtext.data.utils import ngrams_iterator

ImportError: ignored

ERROR: You must give at least one requirement to install (see "pip help install")


In [57]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

ImportError: ignored

In [44]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

NameError: ignored

In [ ]:
# import nltk
# import nltk.translate.bleu_score as bleu
# from nltk.translate.bleu_score import SmoothingFunction
# def bleu_avg(doc, target_doc):
#   bleu_score=[]
#   smoothie = SmoothingFunction().method4
#   for input_sentence, target_sentence in tqdm(zip(doc,target_doc)):
#     try:
#       input_sentence=re.sub('<[^>]*>','',input_sentence)
#       target_sentence=re.sub('<[^>]*>','',target_sentence)
#       result, sentence, attention_plot = evaluate(input_sentence)
#       bleu_score.append(nltk.translate.bleu_score.sentence_bleu(target_sentence.split(' '),result.split(' '),smoothing_function=smoothie))
#     except:
#       pass
#   return np.mean(bleu_score

In [38]:
# import math
# import sys
# from fractions import Fraction
# import warnings
# from collections import Counter
# from nltk.util import ngrams

# def sentence_bleu(
#     references,
#     hypothesis,
#     weights=(0.25, 0.25, 0.25, 0.25),
#     smoothing_function=None,
#     auto_reweigh=False):
#     """
#     Calculate BLEU score (Bilingual Evaluation Understudy) from
#     Papineni, Kishore, Salim Roukos, Todd Ward, and Wei-Jing Zhu. 2002.
#     "BLEU: a method for automatic evaluation of machine translation."
#     In Proceedings of ACL. http://www.aclweb.org/anthology/P02-1040.pdf

#     >>> hypothesis1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
#     ...               'ensures', 'that', 'the', 'military', 'always',
#     ...               'obeys', 'the', 'commands', 'of', 'the', 'party']

#     >>> hypothesis2 = ['It', 'is', 'to', 'insure', 'the', 'troops',
#     ...               'forever', 'hearing', 'the', 'activity', 'guidebook',
#     ...               'that', 'party', 'direct']

#     >>> reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
#     ...               'ensures', 'that', 'the', 'military', 'will', 'forever',
#     ...               'heed', 'Party', 'commands']

#     >>> reference2 = ['It', 'is', 'the', 'guiding', 'principle', 'which',
#     ...               'guarantees', 'the', 'military', 'forces', 'always',
#     ...               'being', 'under', 'the', 'command', 'of', 'the',
#     ...               'Party']

#     >>> reference3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
#     ...               'army', 'always', 'to', 'heed', 'the', 'directions',
#     ...               'of', 'the', 'party']

#     >>> sentence_bleu([reference1, reference2, reference3], hypothesis1) # doctest: +ELLIPSIS
#     0.5045...

#     If there is no ngrams overlap for any order of n-grams, BLEU returns the
#     value 0. This is because the precision for the order of n-grams without
#     overlap is 0, and the geometric mean in the final BLEU score computation
#     multiplies the 0 with the precision of other n-grams. This results in 0
#     (independently of the precision of the othe n-gram orders). The following
#     example has zero 3-gram and 4-gram overlaps:

#     >>> round(sentence_bleu([reference1, reference2, reference3], hypothesis2),4) # doctest: +ELLIPSIS
#     0.0

#     To avoid this harsh behaviour when no ngram overlaps are found a smoothing
#     function can be used.

#     >>> chencherry = SmoothingFunction()
#     >>> sentence_bleu([reference1, reference2, reference3], hypothesis2,
#     ...     smoothing_function=chencherry.method1) # doctest: +ELLIPSIS
#     0.0370...

#     The default BLEU calculates a score for up to 4-grams using uniform
#     weights (this is called BLEU-4). To evaluate your translations with
#     higher/lower order ngrams, use customized weights. E.g. when accounting
#     for up to 5-grams with uniform weights (this is called BLEU-5) use:

#     >>> weights = (1./5., 1./5., 1./5., 1./5., 1./5.)
#     >>> sentence_bleu([reference1, reference2, reference3], hypothesis1, weights) # doctest: +ELLIPSIS
#     0.3920...

#     :param references: reference sentences
#     :type references: list(list(str))
#     :param hypothesis: a hypothesis sentence
#     :type hypothesis: list(str)
#     :param weights: weights for unigrams, bigrams, trigrams and so on
#     :type weights: list(float)
#     :param smoothing_function:
#     :type smoothing_function: SmoothingFunction
#     :param auto_reweigh: Option to re-normalize the weights uniformly.
#     :type auto_reweigh: bool
#     :return: The sentence-level BLEU score.
#     :rtype: float
#     """
#     return corpus_bleu(
#         [references], [hypothesis], weights, smoothing_function, auto_reweigh
#     )



# def corpus_bleu(
#     list_of_references,
#     hypotheses,
#     weights=(0.25, 0.25, 0.25, 0.25),
#     smoothing_function=None,
#     auto_reweigh=False):
#     """
#     Calculate a single corpus-level BLEU score (aka. system-level BLEU) for all
#     the hypotheses and their respective references.

#     Instead of averaging the sentence level BLEU scores (i.e. marco-average
#     precision), the original BLEU metric (Papineni et al. 2002) accounts for
#     the micro-average precision (i.e. summing the numerators and denominators
#     for each hypothesis-reference(s) pairs before the division).

#     >>> hyp1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
#     ...         'ensures', 'that', 'the', 'military', 'always',
#     ...         'obeys', 'the', 'commands', 'of', 'the', 'party']
#     >>> ref1a = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
#     ...          'ensures', 'that', 'the', 'military', 'will', 'forever',
#     ...          'heed', 'Party', 'commands']
#     >>> ref1b = ['It', 'is', 'the', 'guiding', 'principle', 'which',
#     ...          'guarantees', 'the', 'military', 'forces', 'always',
#     ...          'being', 'under', 'the', 'command', 'of', 'the', 'Party']
#     >>> ref1c = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
#     ...          'army', 'always', 'to', 'heed', 'the', 'directions',
#     ...          'of', 'the', 'party']

#     >>> hyp2 = ['he', 'read', 'the', 'book', 'because', 'he', 'was',
#     ...         'interested', 'in', 'world', 'history']
#     >>> ref2a = ['he', 'was', 'interested', 'in', 'world', 'history',
#     ...          'because', 'he', 'read', 'the', 'book']

#     >>> list_of_references = [[ref1a, ref1b, ref1c], [ref2a]]
#     >>> hypotheses = [hyp1, hyp2]
#     >>> corpus_bleu(list_of_references, hypotheses) # doctest: +ELLIPSIS
#     0.5920...

#     The example below show that corpus_bleu() is different from averaging
#     sentence_bleu() for hypotheses

#     >>> score1 = sentence_bleu([ref1a, ref1b, ref1c], hyp1)
#     >>> score2 = sentence_bleu([ref2a], hyp2)
#     >>> (score1 + score2) / 2 # doctest: +ELLIPSIS
#     0.6223...

#     :param list_of_references: a corpus of lists of reference sentences, w.r.t. hypotheses
#     :type list_of_references: list(list(list(str)))
#     :param hypotheses: a list of hypothesis sentences
#     :type hypotheses: list(list(str))
#     :param weights: weights for unigrams, bigrams, trigrams and so on
#     :type weights: list(float)
#     :param smoothing_function:
#     :type smoothing_function: SmoothingFunction
#     :param auto_reweigh: Option to re-normalize the weights uniformly.
#     :type auto_reweigh: bool
#     :return: The corpus-level BLEU score.
#     :rtype: float
#     """
#     # Before proceeding to compute BLEU, perform sanity checks.

#     p_numerators = Counter()  # Key = ngram order, and value = no. of ngram matches.
#     p_denominators = Counter()  # Key = ngram order, and value = no. of ngram in ref.
#     hyp_lengths, ref_lengths = 0, 0

#     assert len(list_of_references) == len(hypotheses), (
#         "The number of hypotheses and their reference(s) should be the " "same "
#     )

#     # Iterate through each hypothesis and their corresponding references.
#     for references, hypothesis in zip(list_of_references, hypotheses):
#         # For each order of ngram, calculate the numerator and
#         # denominator for the corpus-level modified precision.
#         for i, _ in enumerate(weights, start=1):
#             p_i = modified_precision(references, hypothesis, i)
#             p_numerators[i] += p_i.numerator
#             p_denominators[i] += p_i.denominator

#         # Calculate the hypothesis length and the closest reference length.
#         # Adds them to the corpus-level hypothesis and reference counts.
#         hyp_len = len(hypothesis)
#         hyp_lengths += hyp_len
#         ref_lengths += closest_ref_length(references, hyp_len)

#     # Calculate corpus-level brevity penalty.
#     bp = brevity_penalty(ref_lengths, hyp_lengths)

#     # Uniformly re-weighting based on maximum hypothesis lengths if largest
#     # order of n-grams < 4 and weights is set at default.
#     if auto_reweigh:
#         if hyp_lengths < 4 and weights == (0.25, 0.25, 0.25, 0.25):
#             weights = (1 / hyp_lengths,) * hyp_lengths

#     # Collects the various precision values for the different ngram orders.
#     p_n = [
#         Fraction(p_numerators[i], p_denominators[i], _normalize=False)
#         for i, _ in enumerate(weights, start=1)
#     ]

#     # Returns 0 if there's no matching n-grams
#     # We only need to check for p_numerators[1] == 0, since if there's
#     # no unigrams, there won't be any higher order ngrams.
#     if p_numerators[1] == 0:
#         return 0

#     # If there's no smoothing, set use method0 from SmoothinFunction class.
#     if not smoothing_function:
#         smoothing_function = SmoothingFunction().method0
#     # Smoothen the modified precision.
#     # Note: smoothing_function() may convert values into floats;
#     #       it tries to retain the Fraction object as much as the
#     #       smoothing method allows.
#     p_n = smoothing_function(
#         p_n, references=references, hypothesis=hypothesis, hyp_len=hyp_lengths
#     )
#     s = (w_i * math.log(p_i) for w_i, p_i in zip(weights, p_n))
#     s = bp * math.exp(math.fsum(s))
#     return s



# def modified_precision(references, hypothesis, n):
#     """
#     Calculate modified ngram precision.

#     The normal precision method may lead to some wrong translations with
#     high-precision, e.g., the translation, in which a word of reference
#     repeats several times, has very high precision.

#     This function only returns the Fraction object that contains the numerator
#     and denominator necessary to calculate the corpus-level precision.
#     To calculate the modified precision for a single pair of hypothesis and
#     references, cast the Fraction object into a float.

#     The famous "the the the ... " example shows that you can get BLEU precision
#     by duplicating high frequency words.

#         >>> reference1 = 'the cat is on the mat'.split()
#         >>> reference2 = 'there is a cat on the mat'.split()
#         >>> hypothesis1 = 'the the the the the the the'.split()
#         >>> references = [reference1, reference2]
#         >>> float(modified_precision(references, hypothesis1, n=1)) # doctest: +ELLIPSIS
#         0.2857...

#     In the modified n-gram precision, a reference word will be considered
#     exhausted after a matching hypothesis word is identified, e.g.

#         >>> reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
#         ...               'ensures', 'that', 'the', 'military', 'will',
#         ...               'forever', 'heed', 'Party', 'commands']
#         >>> reference2 = ['It', 'is', 'the', 'guiding', 'principle', 'which',
#         ...               'guarantees', 'the', 'military', 'forces', 'always',
#         ...               'being', 'under', 'the', 'command', 'of', 'the',
#         ...               'Party']
#         >>> reference3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
#         ...               'army', 'always', 'to', 'heed', 'the', 'directions',
#         ...               'of', 'the', 'party']
#         >>> hypothesis = 'of the'.split()
#         >>> references = [reference1, reference2, reference3]
#         >>> float(modified_precision(references, hypothesis, n=1))
#         1.0
#         >>> float(modified_precision(references, hypothesis, n=2))
#         1.0

#     An example of a normal machine translation hypothesis:

#         >>> hypothesis1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
#         ...               'ensures', 'that', 'the', 'military', 'always',
#         ...               'obeys', 'the', 'commands', 'of', 'the', 'party']

#         >>> hypothesis2 = ['It', 'is', 'to', 'insure', 'the', 'troops',
#         ...               'forever', 'hearing', 'the', 'activity', 'guidebook',
#         ...               'that', 'party', 'direct']

#         >>> reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
#         ...               'ensures', 'that', 'the', 'military', 'will',
#         ...               'forever', 'heed', 'Party', 'commands']

#         >>> reference2 = ['It', 'is', 'the', 'guiding', 'principle', 'which',
#         ...               'guarantees', 'the', 'military', 'forces', 'always',
#         ...               'being', 'under', 'the', 'command', 'of', 'the',
#         ...               'Party']

#         >>> reference3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
#         ...               'army', 'always', 'to', 'heed', 'the', 'directions',
#         ...               'of', 'the', 'party']
#         >>> references = [reference1, reference2, reference3]
#         >>> float(modified_precision(references, hypothesis1, n=1)) # doctest: +ELLIPSIS
#         0.9444...
#         >>> float(modified_precision(references, hypothesis2, n=1)) # doctest: +ELLIPSIS
#         0.5714...
#         >>> float(modified_precision(references, hypothesis1, n=2)) # doctest: +ELLIPSIS
#         0.5882352941176471
#         >>> float(modified_precision(references, hypothesis2, n=2)) # doctest: +ELLIPSIS
#         0.07692...


#     :param references: A list of reference translations.
#     :type references: list(list(str))
#     :param hypothesis: A hypothesis translation.
#     :type hypothesis: list(str)
#     :param n: The ngram order.
#     :type n: int
#     :return: BLEU's modified precision for the nth order ngram.
#     :rtype: Fraction
#     """
#     # Extracts all ngrams in hypothesis
#     # Set an empty Counter if hypothesis is empty.
#     counts = Counter(ngrams(hypothesis, n)) if len(hypothesis) >= n else Counter()
#     # Extract a union of references' counts.
#     # max_counts = reduce(or_, [Counter(ngrams(ref, n)) for ref in references])
#     max_counts = {}
#     for reference in references:
#         reference_counts = (
#             Counter(ngrams(reference, n)) if len(reference) >= n else Counter()
#         )
#         for ngram in counts:
#             max_counts[ngram] = max(max_counts.get(ngram, 0), reference_counts[ngram])

#     # Assigns the intersection between hypothesis and references' counts.
#     clipped_counts = {
#         ngram: min(count, max_counts[ngram]) for ngram, count in counts.items()
#     }

#     numerator = sum(clipped_counts.values())
#     # Ensures that denominator is minimum 1 to avoid ZeroDivisionError.
#     # Usually this happens when the ngram order is > len(reference).
#     denominator = max(1, sum(counts.values()))

#     return Fraction(numerator, denominator, _normalize=False)



# def closest_ref_length(references, hyp_len):
#     """
#     This function finds the reference that is the closest length to the
#     hypothesis. The closest reference length is referred to as *r* variable
#     from the brevity penalty formula in Papineni et. al. (2002)

#     :param references: A list of reference translations.
#     :type references: list(list(str))
#     :param hyp_len: The length of the hypothesis.
#     :type hyp_len: int
#     :return: The length of the reference that's closest to the hypothesis.
#     :rtype: int
#     """
#     ref_lens = (len(reference) for reference in references)
#     closest_ref_len = min(
#         ref_lens, key=lambda ref_len: (abs(ref_len - hyp_len), ref_len)
#     )
#     return closest_ref_len



# def brevity_penalty(closest_ref_len, hyp_len):
#     """
#     Calculate brevity penalty.

#     As the modified n-gram precision still has the problem from the short
#     length sentence, brevity penalty is used to modify the overall BLEU
#     score according to length.

#     An example from the paper. There are three references with length 12, 15
#     and 17. And a concise hypothesis of the length 12. The brevity penalty is 1.

#         >>> reference1 = list('aaaaaaaaaaaa')      # i.e. ['a'] * 12
#         >>> reference2 = list('aaaaaaaaaaaaaaa')   # i.e. ['a'] * 15
#         >>> reference3 = list('aaaaaaaaaaaaaaaaa') # i.e. ['a'] * 17
#         >>> hypothesis = list('aaaaaaaaaaaa')      # i.e. ['a'] * 12
#         >>> references = [reference1, reference2, reference3]
#         >>> hyp_len = len(hypothesis)
#         >>> closest_ref_len =  closest_ref_length(references, hyp_len)
#         >>> brevity_penalty(closest_ref_len, hyp_len)
#         1.0

#     In case a hypothesis translation is shorter than the references, penalty is
#     applied.

#         >>> references = [['a'] * 28, ['a'] * 28]
#         >>> hypothesis = ['a'] * 12
#         >>> hyp_len = len(hypothesis)
#         >>> closest_ref_len =  closest_ref_length(references, hyp_len)
#         >>> brevity_penalty(closest_ref_len, hyp_len)
#         0.2635971381157267

#     The length of the closest reference is used to compute the penalty. If the
#     length of a hypothesis is 12, and the reference lengths are 13 and 2, the
#     penalty is applied because the hypothesis length (12) is less then the
#     closest reference length (13).

#         >>> references = [['a'] * 13, ['a'] * 2]
#         >>> hypothesis = ['a'] * 12
#         >>> hyp_len = len(hypothesis)
#         >>> closest_ref_len =  closest_ref_length(references, hyp_len)
#         >>> brevity_penalty(closest_ref_len, hyp_len) # doctest: +ELLIPSIS
#         0.9200...

#     The brevity penalty doesn't depend on reference order. More importantly,
#     when two reference sentences are at the same distance, the shortest
#     reference sentence length is used.

#         >>> references = [['a'] * 13, ['a'] * 11]
#         >>> hypothesis = ['a'] * 12
#         >>> hyp_len = len(hypothesis)
#         >>> closest_ref_len =  closest_ref_length(references, hyp_len)
#         >>> bp1 = brevity_penalty(closest_ref_len, hyp_len)
#         >>> hyp_len = len(hypothesis)
#         >>> closest_ref_len =  closest_ref_length(reversed(references), hyp_len)
#         >>> bp2 = brevity_penalty(closest_ref_len, hyp_len)
#         >>> bp1 == bp2 == 1
#         True

#     A test example from mteval-v13a.pl (starting from the line 705):

#         >>> references = [['a'] * 11, ['a'] * 8]
#         >>> hypothesis = ['a'] * 7
#         >>> hyp_len = len(hypothesis)
#         >>> closest_ref_len =  closest_ref_length(references, hyp_len)
#         >>> brevity_penalty(closest_ref_len, hyp_len) # doctest: +ELLIPSIS
#         0.8668...

#         >>> references = [['a'] * 11, ['a'] * 8, ['a'] * 6, ['a'] * 7]
#         >>> hypothesis = ['a'] * 7
#         >>> hyp_len = len(hypothesis)
#         >>> closest_ref_len =  closest_ref_length(references, hyp_len)
#         >>> brevity_penalty(closest_ref_len, hyp_len)
#         1.0

#     :param hyp_len: The length of the hypothesis for a single sentence OR the
#     sum of all the hypotheses' lengths for a corpus
#     :type hyp_len: int
#     :param closest_ref_len: The length of the closest reference for a single
#     hypothesis OR the sum of all the closest references for every hypotheses.
#     :type closest_ref_len: int
#     :return: BLEU's brevity penalty.
#     :rtype: float
#     """
#     if hyp_len > closest_ref_len:
#         return 1
#     # If hypothesis is empty, brevity penalty = 0 should result in BLEU = 0.0
#     elif hyp_len == 0:
#         return 0
#     else:
#         return math.exp(1 - closest_ref_len / hyp_len)



# class SmoothingFunction:
#     """
#     This is an implementation of the smoothing techniques
#     for segment-level BLEU scores that was presented in
#     Boxing Chen and Collin Cherry (2014) A Systematic Comparison of
#     Smoothing Techniques for Sentence-Level BLEU. In WMT14.
#     http://acl2014.org/acl2014/W14-33/pdf/W14-3346.pdf
#     """

#     def __init__(self, epsilon=0.1, alpha=5, k=5):
#         """
#         This will initialize the parameters required for the various smoothing
#         techniques, the default values are set to the numbers used in the
#         experiments from Chen and Cherry (2014).

#         >>> hypothesis1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which', 'ensures',
#         ...                 'that', 'the', 'military', 'always', 'obeys', 'the',
#         ...                 'commands', 'of', 'the', 'party']
#         >>> reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that', 'ensures',
#         ...               'that', 'the', 'military', 'will', 'forever', 'heed',
#         ...               'Party', 'commands']

#         >>> chencherry = SmoothingFunction()
#         >>> print(sentence_bleu([reference1], hypothesis1)) # doctest: +ELLIPSIS
#         0.4118...
#         >>> print(sentence_bleu([reference1], hypothesis1, smoothing_function=chencherry.method0)) # doctest: +ELLIPSIS
#         0.4118...
#         >>> print(sentence_bleu([reference1], hypothesis1, smoothing_function=chencherry.method1)) # doctest: +ELLIPSIS
#         0.4118...
#         >>> print(sentence_bleu([reference1], hypothesis1, smoothing_function=chencherry.method2)) # doctest: +ELLIPSIS
#         0.4489...
#         >>> print(sentence_bleu([reference1], hypothesis1, smoothing_function=chencherry.method3)) # doctest: +ELLIPSIS
#         0.4118...
#         >>> print(sentence_bleu([reference1], hypothesis1, smoothing_function=chencherry.method4)) # doctest: +ELLIPSIS
#         0.4118...
#         >>> print(sentence_bleu([reference1], hypothesis1, smoothing_function=chencherry.method5)) # doctest: +ELLIPSIS
#         0.4905...
#         >>> print(sentence_bleu([reference1], hypothesis1, smoothing_function=chencherry.method6)) # doctest: +ELLIPSIS
#         0.4135...
#         >>> print(sentence_bleu([reference1], hypothesis1, smoothing_function=chencherry.method7)) # doctest: +ELLIPSIS
#         0.4905...

#         :param epsilon: the epsilon value use in method 1
#         :type epsilon: float
#         :param alpha: the alpha value use in method 6
#         :type alpha: int
#         :param k: the k value use in method 4
#         :type k: int
#         """
#         self.epsilon = epsilon
#         self.alpha = alpha
#         self.k = k

#     def method0(self, p_n, *args, **kwargs):
#             """
#             No smoothing.
#             """
#             p_n_new = []
#             for i, p_i in enumerate(p_n):
#                 if p_i.numerator != 0:
#                     p_n_new.append(p_i)
#                 else:
#                     _msg = str(
#                         "\nThe hypothesis contains 0 counts of {}-gram overlaps.\n"
#                         "Therefore the BLEU score evaluates to 0, independently of\n"
#                         "how many N-gram overlaps of lower order it contains.\n"
#                         "Consider using lower n-gram order or use "
#                         "SmoothingFunction()"
#                     ).format(i + 1)
#                     warnings.warn(_msg)
#                     # When numerator==0 where denonminator==0 or !=0, the result
#                     # for the precision score should be equal to 0 or undefined.
#                     # Due to BLEU geometric mean computation in logarithm space,
#                     # we we need to take the return sys.float_info.min such that
#                     # math.log(sys.float_info.min) returns a 0 precision score.
#                     p_n_new.append(sys.float_info.min)
#             return p_n_new


#     def method1(self, p_n, *args, **kwargs):
#             """
#             Smoothing method 1: Add *epsilon* counts to precision with 0 counts.
#             """
#             return [
#                 (p_i.numerator + self.epsilon) / p_i.denominator
#                 if p_i.numerator == 0
#                 else p_i
#                 for p_i in p_n
#             ]


#     def method2(self, p_n, *args, **kwargs):
#             """
#             Smoothing method 2: Add 1 to both numerator and denominator from
#             Chin-Yew Lin and Franz Josef Och (2004) Automatic evaluation of
#             machine translation quality using longest common subsequence and
#             skip-bigram statistics. In ACL04.
#             """
#             return [
#                 Fraction(p_i.numerator + 1, p_i.denominator + 1, _normalize=False)
#                 for p_i in p_n
#             ]


#     def method3(self, p_n, *args, **kwargs):
#             """
#             Smoothing method 3: NIST geometric sequence smoothing
#             The smoothing is computed by taking 1 / ( 2^k ), instead of 0, for each
#             precision score whose matching n-gram count is null.
#             k is 1 for the first 'n' value for which the n-gram match count is null/
#             For example, if the text contains:
#             - one 2-gram match
#             - and (consequently) two 1-gram matches
#             the n-gram count for each individual precision score would be:
#             - n=1  =>  prec_count = 2     (two unigrams)
#             - n=2  =>  prec_count = 1     (one bigram)
#             - n=3  =>  prec_count = 1/2   (no trigram,  taking 'smoothed' value of 1 / ( 2^k ), with k=1)
#             - n=4  =>  prec_count = 1/4   (no fourgram, taking 'smoothed' value of 1 / ( 2^k ), with k=2)
#             """
#             incvnt = 1  # From the mteval-v13a.pl, it's referred to as k.
#             for i, p_i in enumerate(p_n):
#                 if p_i.numerator == 0:
#                     p_n[i] = 1 / (2 ** incvnt * p_i.denominator)
#                     incvnt += 1
#             return p_n


#     def method4(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
#             """
#             Smoothing method 4:
#             Shorter translations may have inflated precision values due to having
#             smaller denominators; therefore, we give them proportionally
#             smaller smoothed counts. Instead of scaling to 1/(2^k), Chen and Cherry
#             suggests dividing by 1/ln(len(T)), where T is the length of the translation.
#             """
#             hyp_len = hyp_len if hyp_len else len(hypothesis)
#             for i, p_i in enumerate(p_n):
#                 if p_i.numerator == 0 and hyp_len != 0:
#                     incvnt = i + 1 * self.k / math.log(
#                         hyp_len
#                     )  # Note that this K is different from the K from NIST.
#                     p_n[i] = incvnt / p_i.denominator
#             return p_n


#     def method5(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
#             """
#             Smoothing method 5:
#             The matched counts for similar values of n should be similar. To a
#             calculate the n-gram matched count, it averages the n−1, n and n+1 gram
#             matched counts.
#             """
#             hyp_len = hyp_len if hyp_len else len(hypothesis)
#             m = {}
#             # Requires an precision value for an addition ngram order.
#             p_n_plus1 = p_n + [modified_precision(references, hypothesis, 5)]
#             m[-1] = p_n[0] + 1
#             for i, p_i in enumerate(p_n):
#                 p_n[i] = (m[i - 1] + p_i + p_n_plus1[i + 1]) / 3
#                 m[i] = p_n[i]
#             return p_n


#     def method6(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
#             """
#             Smoothing method 6:
#             Interpolates the maximum likelihood estimate of the precision *p_n* with
#             a prior estimate *pi0*. The prior is estimated by assuming that the ratio
#             between pn and pn−1 will be the same as that between pn−1 and pn−2; from
#             Gao and He (2013) Training MRF-Based Phrase Translation Models using
#             Gradient Ascent. In NAACL.
#             """
#             hyp_len = hyp_len if hyp_len else len(hypothesis)
#             # This smoothing only works when p_1 and p_2 is non-zero.
#             # Raise an error with an appropriate message when the input is too short
#             # to use this smoothing technique.
#             assert p_n[2], "This smoothing method requires non-zero precision for bigrams."
#             for i, p_i in enumerate(p_n):
#                 if i in [0, 1]:  # Skips the first 2 orders of ngrams.
#                     continue
#                 else:
#                     pi0 = 0 if p_n[i - 2] == 0 else p_n[i - 1] ** 2 / p_n[i - 2]
#                     # No. of ngrams in translation that matches the reference.
#                     m = p_i.numerator
#                     # No. of ngrams in translation.
#                     l = sum(1 for _ in ngrams(hypothesis, i + 1))
#                     # Calculates the interpolated precision.
#                     p_n[i] = (m + self.alpha * pi0) / (l + self.alpha)
#             return p_n


#     def method7(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
#             """
#             Smoothing method 7:
#             Interpolates methods 5 and 6.
#             """
#             hyp_len = hyp_len if hyp_len else len(hypothesis)
#             p_n = self.method4(p_n, references, hypothesis, hyp_len)
#             p_n = self.method5(p_n, references, hypothesis, hyp_len)
#             return p_n